# Cleaning Additional datasets

In [6]:
import pandas as pd
import numpy as np

## Data cleaning: Trees dataset

In [3]:
df_trees = pd.read_csv("data_trees/new_york_tree_census_2015.csv")
df_trees.head(2)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,606945,305778,2016-06-28,10,0,OnCurb,Alive,Good,Fraxinus pennsylvanica,green ash,...,25,14,QN37,Kew Gardens Hills,4125700,New York,40.724339,-73.805180,1038250.055,203232.9417
1,160321,341273,2015-08-19,9,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,34,13,QN28,Jackson Heights,4030902,New York,40.756626,-73.894167,1013570.588,214953.6472


In [4]:
print("Original: ", df_trees.shape)
print(df_trees.memory_usage(index=True).sum()/1E6)

Original:  (683788, 41)
224.282592


Since this dataset was so large we decided to do a couple of different steps of data cleaning. First we removed all trees that were noted to have some kind of problems, not be alive or have a diameter of 0 inches. Then we created a new column of the dataset with the diamaeter of the trees given in centimeters. Then we droped all entries with missing values and removed irrelevant columns.

In [7]:
df_trees = df_trees[(df_trees.problems == 'None')]
df_trees = df_trees[(df_trees.status == 'Alive')]
df_trees = df_trees[df_trees.tree_dbh!=0]
df_trees['diameter_cm'] = np.round(df_trees['tree_dbh']*2.54,2) #to SI units (meters)
df_trees = df_trees[['spc_common', 'diameter_cm', 'latitude', 'longitude']].dropna(axis=0)
df_trees = df_trees.reset_index(drop=True)
print("After cleaning:", df_trees.shape)
print(df_trees.memory_usage(index=True).sum()/1E6)

After cleaning: (426086, 4)
13.63488


Even after the data cleaning the dataset was still too large for us to use without issues, so we decided to downsample it, removing half of the remaining entries.

In [8]:
df_trees_size = df_trees.shape[0]
df_trees_sample = df_trees.sample(round(df_trees_size/2), random_state=33)
print("After downsampling: ", df_trees_sample.shape)
print(df_trees_sample.memory_usage(index=True).sum()/1E6)

After downsampling:  (213043, 4)
8.52172


In [10]:
df_trees = df_trees_sample
df_trees.head(2)

,spc_common,diameter_cm,latitude,longitude
396538,green ash,38.10,40.774001,-73.748105
6700,littleleaf linden,5.08,40.572681,-74.119011


<font color='darkred'>Note:</font> The cell below is disabled to avoid overwriting files

In [11]:
#df_trees.to_csv(r'data_trees/trees_data_clean.csv', index=False)

## Data cleaning: Rats dataset

In [17]:
df_rats = pd.read_csv("data_rats/Rat_Sightings.csv")
df_rats.head(2)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,31464015,09/04/2015 12:00:00 AM,09/18/2015 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,3+ Family Mixed Use Building,10006.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.707772,-74.012963,"(40.70777155363643, -74.01296309970473)"
1,31464024,09/04/2015 12:00:00 AM,10/28/2015 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Commercial Building,10306.0,2270 HYLAN BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.575209,-74.104547,"(40.575209242947444, -74.1045465185469)"


In [18]:
print("Original length: " , df_rats.shape[0])

Original length:  101914


For the rats dataset we decided to only use entries from 2017 since we were worried too old data might impach the data analysis negatively. Here we also droped all irrelevant features.

In [19]:
df_rats['Created Date'] = pd.to_datetime(df_rats['Created Date'])
df_rats = df_rats[df_rats['Created Date'].dt.year > 2016] #Only take the ones from 2017
df_rats = df_rats[['Latitude', 'Longitude', 'Location Type']].dropna(axis=0)
df_rats = df_rats.rename(columns={'Latitude': 'latitude', 'Longitude':'longitude'})
df_rats = df_rats.reset_index(drop=True)
print("Final length: ", df_rats.shape[0])
df_rats.head(2)

Final length:  14307


,latitude,longitude,Location Type
0,40.68123,-73.97288,3+ Family Apt. Building
1,40.68123,-73.97288,3+ Family Apt. Building


<font color='darkred'>Note:</font> The cell below is disabled to avoid overwriting files

In [20]:
#df_rats.to_csv(r'data_rats/rats_data_clean.csv', index=False)

## Data cleaning: Tourist Places dataset

In [23]:
from geopy.geocoders import Photon

In [ ]:
g = Photon()
NYC = g.geocode("New York City")
NYC_coor  = [NYC.latitude, NYC.longitude]

In [26]:
df_places = pd.read_excel("data_places/New_York_Tourist_Locations.xlsx")
df_places.head(2)

,Tourist_Spot,Address,Zipcode
0,47th Street - The Diamond District,"47th St. betw. Fifth & Sixth Aves. Manhattan, ...",10036
1,9/11 Memorial & Museum,World Trade Center (museum: 180 Greenwich St.)...,10007


In [27]:
print("Number of rows: ", len(df_places.Address))

Number of rows:  347


The dataset with the tourist attractions did not contain the latitude and longtitude, and thus we had to retreve the coordinates from the address or the name of the attraction. We did using Photon form geopy.

In [28]:
# Data prep
pd.set_option("max_colwidth", 400)
df_places_parse = df_places.copy(deep=True)

df_places_parse.Address.replace({'betw. ':'between ','Rd. ':'Road ', 'Blvd. ':'Boulevard ', 'Ave. ':'Avenue ',
                                                'Aves. ':'Avenues ','St. ':'Street ', ' at':'', 'W.':'', 'W':''}, regex=True, inplace=True)
df_places_parse.Address = df_places_parse.Address.str.split(', NY').str[0]
df_places_parse.Address = df_places_parse.Address.str.split('NY').str[0]
df_places_parse.Address = df_places_parse.Address.str.split('mail').str[0]
df_places_parse.Address.replace('',np.nan)
df_places_parse.Address.dropna(inplace=True)
df_places_parse.head(3)

,Tourist_Spot,Address,Zipcode
0,47th Street - The Diamond District,47th Street between Fifth & Sixth Avenues Manhattan,10036
1,9/11 Memorial & Museum,rld Trade Center (museum: 180 Greenwich St.) Manhattan,10007
2,Abyssinian Baptist Church,132 Odell Clark Place New York,10030


In [29]:
#Search location from Address column and write it in 'location' column. If not successful, write 0
df_places_parse_addr = df_places_parse.copy(deep=True)
def addres_to_loc(addr):
    a = g.geocode(addr, location_bias=NYC_coor)
    if a is not None:
        return a
    else:
        return 0

df_places_parse_addr['location'] = df_places_parse_addr.Address.apply(lambda row: addres_to_loc(row))
df_places_parse_addr.head(3)

,Tourist_Spot,Address,Zipcode,location
0,47th Street - The Diamond District,47th Street between Fifth & Sixth Avenues Manhattan,10036,0
1,9/11 Memorial & Museum,rld Trade Center (museum: 180 Greenwich St.) Manhattan,10007,"(National September 11 Memorial & Museum, 180, Greenwich Street, 10007, Greenwich Street, New York, NY, United States, (40.711304049999995, -74.0132825413914))"
2,Abyssinian Baptist Church,132 Odell Clark Place New York,10030,"(Abyssinian Baptist Church, 132, Odell M Clarke Place, 10030, Odell M Clarke Place, New York, New York, United States, (40.8166357, -73.941477910502))"


In [30]:
#Dataframe is split in two, for right and wrong results
df_places_parse_addr_0 = df_places_parse_addr[df_places_parse_addr.location==0].copy(deep=True)
df_places_parse_addr = df_places_parse_addr[df_places_parse_addr.location !=0]
# Locations left to process
print("Locations left to process: ", df_places_parse_addr_0.shape[0])

Locations left to process:  91


In [31]:
#This time, search location from Tourist_Spot column (in wrong results dataframe) and write it 
#in column location. If not successful, write 0
df_places_parse_spot = df_places_parse_addr_0.copy(deep=True)
def get_loc_spot(addr):
    a = g.geocode(addr, location_bias=NYC_coor)
    if a is not None:
        return a
    else:
        return 0

df_places_parse_spot['location'] = df_places_parse_spot.Tourist_Spot.apply(lambda row: get_loc_spot(row))
df_places_parse_spot.head(2)

,Tourist_Spot,Address,Zipcode,location
0,47th Street - The Diamond District,47th Street between Fifth & Sixth Avenues Manhattan,10036,"(Diamond District, West 47th Street, 10036, West 47th Street, New York, New York, United States, (40.757242, -73.9801423))"
12,Bateaux New York,"Pier 61, Chelsea Piers (23rd Street & Hudson River) Manhattan",10011,"(Bateaux Street, 13440, City of Rome, New York, United States, (43.2299737, -75.4931233))"


In [32]:
# Dataframe is split in two, for right and wrong results
df_places_parse_spot_0 = df_places_parse_spot[df_places_parse_spot.location==0].copy(deep=True)
df_places_parse_spot =  df_places_parse_spot[df_places_parse_spot.location !=0]
# Locations left to process
print("Locations left to process: ", df_places_parse_spot_0.shape[0])

Locations left to process:  10


In [33]:
df_places_manually = df_places_parse_spot_0.copy(deep=True)
indexNames = df_places_manually[ (df_places_manually['Address'] == '')].index
df_places_manually.drop(indexNames, inplace=True)
df_places_manually.head(2)

,Tourist_Spot,Address,Zipcode,location
26,Central Park Conservancy—Official Central Park Tours,"14 E. 60th St., 8th fl. New York",10022,0
27,Charles A. Dana Discovery Center/Harlem Meer,106th to 110th Street on the east side of the park Manhattan,10029,0


In [35]:
df_places_manually.iloc[0].location = g.geocode('Central Park Conservancy Park', location_bias=NYC_coor)
df_places_manually.iloc[1].location = g.geocode('Charles A. Dana Discovery Center', location_bias=NYC_coor)
df_places_manually.iloc[2].location = g.geocode('Sea, Air and Space Museum', location_bias=NYC_coor)
df_places_manually.iloc[3].location = g.geocode('Information Center Pier 15', location_bias=NYC_coor)
df_places_manually.iloc[4].location = g.geocode('Official Information Center Times Square', location_bias=NYC_coor)
df_places_manually.iloc[5].location = g.geocode('Denos Wonder Wheel Amusement Park', location_bias=NYC_coor)
df_places_manually.iloc[6].location = g.geocode('1 FDR Four Freedoms Park', location_bias=NYC_coor)
df_places_manually.iloc[7].location = g.geocode('UNICEF House ', location_bias=NYC_coor)
df_places_manually.iloc[8].location = g.geocode('Visitor Center, 31 Chambers Street', location_bias=NYC_coor)

df_places_manually.head(2)

,Tourist_Spot,Address,Zipcode,location
26,Central Park Conservancy—Official Central Park Tours,"14 E. 60th St., 8th fl. New York",10022,"(Central Park, United States, (40.7827725, -73.9653627406542))"
27,Charles A. Dana Discovery Center/Harlem Meer,106th to 110th Street on the east side of the park Manhattan,10029,"(Charles Ives Center for the Performing Arts, University Boulevard, 96810, University Boulevard, Danbury, CT, United States, (41.3948169, -73.4940125))"


In [36]:
df_places_parsed = df_places_parse_addr.append([df_places_parse_spot, df_places_manually])
df_places_parsed.reset_index(inplace=True, drop=True)
# Create latitude and longitude columns from location column
df_places_parsed['latitude'] = df_places_parsed.location.apply(lambda x: x.latitude)
df_places_parsed['longitude'] = df_places_parsed.location.apply(lambda x: x.longitude)
df_places_parsed.head(2)

,Tourist_Spot,Address,Zipcode,location,latitude,longitude
0,9/11 Memorial & Museum,rld Trade Center (museum: 180 Greenwich St.) Manhattan,10007,"(National September 11 Memorial & Museum, 180, Greenwich Street, 10007, Greenwich Street, New York, NY, United States, (40.711304049999995, -74.0132825413914))",40.711304,-74.013283
1,Abyssinian Baptist Church,132 Odell Clark Place New York,10030,"(Abyssinian Baptist Church, 132, Odell M Clarke Place, 10030, Odell M Clarke Place, New York, New York, United States, (40.8166357, -73.941477910502))",40.816636,-73.941478


In [37]:
print(df_places_parsed.shape[0])
df_places_parsed_filtered = df_places_parsed[
                                             (df_places_parsed.latitude>40.2) &
                                             (df_places_parsed.longitude>-74.15) &
                                             (df_places_parsed.latitude<44) &
                                             (df_places_parsed.longitude<-73.7) ].reset_index(drop=True)

print(df_places_parsed_filtered.shape[0])
df_discard = df_places_parsed[~(
                                             (df_places_parsed.latitude>40.2) &
                                             (df_places_parsed.longitude>-74.15) &
                                             (df_places_parsed.latitude<44) &
                                             (df_places_parsed.longitude<-73.7)) ].reset_index(drop=True)
print("Discarded: ", df_discard.shape[0])

346
314
Discarded:  32


<font color='darkred'>Note:</font> The cell below is disabled to avoid overwriting files

In [38]:
#df_places_parsed_filtered.to_csv(r'data_places/places_data_clean.csv', index=False)